In [1]:
import pandas as pd
import numpy as np

In [32]:
close_returns_raw = pd.read_csv('dataset/close_returns.csv', index_col = 0, parse_dates = True)
high_prices_raw = pd.read_csv('dataset/high_prices.csv', index_col = 0, parse_dates = True)
low_prices_raw = pd.read_csv('dataset/low_prices.csv', index_col = 0, parse_dates = True)
volume_raw = pd.read_csv('dataset/volume.csv', index_col = 0, parse_dates = True)
signals_A_raw = pd.read_csv('dataset/signals_A.csv', index_col = 0, parse_dates = True)
signals_B_raw = pd.read_csv('dataset/signals_B.csv', index_col = 0, parse_dates = True)

In [33]:
close_returns_raw.head()

,A.0,A.1,A.2,A.3,A.4,A.5,A.6,A.7,A.8,A.9,...,A.990,A.991,A.992,A.993,A.994,A.995,A.996,A.997,A.998,A.999
time,,,,,,,,,,,,,,,,,,,,,
2017-01-03 09:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-03 09:45:00,-0.010530,0.002627,-0.005865,-0.002815,-0.006875,0.004111,-0.007562,0.034228,-0.016570,-0.023604,...,0.002529,-0.005074,0.000942,0.005411,NaN,-0.002108,0.006905,-0.004139,0.006602,0.002468
2017-01-03 10:00:00,-0.005218,-0.003813,0.006697,-0.006235,-0.001562,0.030088,0.017317,-0.014377,0.002730,-0.003007,...,-0.001575,NaN,0.019034,-0.003286,NaN,-0.009330,NaN,0.004126,0.001973,0.023038
2017-01-03 10:15:00,0.004456,0.007341,0.006147,-0.001772,-0.001146,-0.018533,-0.015283,0.028084,-0.015930,0.001107,...,0.001118,NaN,-0.003259,-0.004713,NaN,-0.000681,NaN,0.008012,0.006369,-0.019062
2017-01-03 10:30:00,-0.001799,-0.010040,0.008337,-0.015490,0.000895,-0.022239,0.011906,NaN,0.034934,-0.008240,...,0.005739,NaN,-0.002320,-0.003620,NaN,-0.001107,0.007323,-0.018825,-0.004333,0.000031


In [16]:
#def train_test_split_and_fillna(dataframe):
#    # Fill forward 
#    train_data = dataframe[:4360]
#    test_data = dataframe[4360:]   

'2017-08-31 15:45:00'

In [35]:
median_volume_each_stock = volume_raw.median(axis = 0)
volume_normalised = volume_raw.apply(lambda x: x / median_volume_each_stock, axis=1)

In [36]:
volatility = (high_prices_raw - low_prices_raw) / low_prices_raw

In [75]:
def intra_day_forward_fillna_max_filling_1_hour_gap(dataframe):
    return dataframe.groupby(dataframe.index.dayofyear).apply(
        lambda x: x.fillna(method = "ffill", axis = 0, limit = 4))

In [76]:
volume_normalised_filled = intra_day_forward_fillna_max_filling_1_hour_gap(volume_normalised)
volatility_filled = intra_day_forward_fillna_max_filling_1_hour_gap(volatility)
signal_A_raw_filled = intra_day_forward_fillna_max_filling_1_hour_gap(signals_A_raw)
signal_B_raw_filled = intra_day_forward_fillna_max_filling_1_hour_gap(signals_B_raw)

In [82]:
close_returns_raw_filled = close_returns_raw.groupby(close_returns_raw.index.dayofyear).apply(
    lambda x: x.fillna(value = 0.0, axis = 0)) 
# Different from "ffill", if use "limit" here, it would specify maximum 
# total number of 0 that can be used to pad each col. Not what we want.

In [101]:
list_of_df_each_for_one_stock = [pd.concat([close_returns_raw_filled.iloc[:, a],
                                            volume_normalised_filled.iloc[:, a],
                                            volatility_filled.iloc[:, a],
                                            signal_A_raw_filled.iloc[:, a],
                                            signal_B_raw_filled.iloc[:, a]], axis = 1)
                                 for a in range(1000)] # Total 1000 stocks

In [109]:
for e in list_of_df_each_for_one_stock:
    e.columns = ["return", "volume", "volatility", "signal_A", "signal_B"]               

In [113]:
list_of_df_each_for_one_stock[0].head()

,return,volume,volatility,signal_A,signal_B
time,,,,,
2017-01-03 09:30:00,0.000000,1.357828,0.011823,NaN,NaN
2017-01-03 09:45:00,-0.010530,0.840949,0.003595,-0.012937,NaN
2017-01-03 10:00:00,-0.005218,1.191392,0.005506,-0.004742,NaN
2017-01-03 10:15:00,0.004456,0.554745,0.004171,-0.001138,NaN
2017-01-03 10:30:00,-0.001799,0.484170,0.004360,0.003315,NaN


In [217]:
test = list_of_df_each_for_one_stock[0].groupby(list_of_df_each_for_one_stock[0].index.dayofyear)

In [124]:
test1 = [test.get_group(x) for x in test.groups]

In [190]:
[e for e in test1 if len(e.index) < 26]

[                       return    volume  volatility  signal_A  signal_B
 time                                                                   
 2017-07-03 09:30:00  0.018765  1.487080    0.005106  0.015306  1.182464
 2017-07-03 09:45:00 -0.000549  1.040039    0.002290 -0.000209  0.841115
 2017-07-03 10:00:00 -0.011013  1.044533    0.003738 -0.007487  0.848956
 2017-07-03 10:15:00  0.005084  0.782640    0.004426  0.005433  0.628659
 2017-07-03 10:30:00 -0.011877  1.742122    0.003587 -0.013553  1.340798
 2017-07-03 10:45:00  0.015509  0.992707    0.001381  0.013264  0.762829
 2017-07-03 11:00:00  0.002625  1.198354    0.003234  0.001094  0.925794
 2017-07-03 11:15:00 -0.010720  0.756174    0.001786 -0.007769  0.596615
 2017-07-03 11:30:00  0.005567  0.694919    0.002379  0.002923  0.555887
 2017-07-03 11:45:00  0.013474  1.037590    0.003729  0.013477  0.818048
 2017-07-03 12:00:00 -0.014437  0.710794    0.001864 -0.010423  0.564153
 2017-07-03 12:15:00  0.010409  0.916773    0.00110

In [223]:
test2 = [x.groupby(x.index.dayofyear) for x in list_of_df_each_for_one_stock]

In [225]:
test3 = [x.get_group(a) for x in test2 for a in x.groups]

In [236]:
test4 = test3[0:2]

In [237]:
test5 = [make_more(e) for e in test4]

In [238]:
test5

[[                       return    volume  volatility  signal_A  signal_B
  time                                                                   
  2017-01-03 09:30:00  0.000000  1.357828    0.011823       NaN       NaN
  2017-01-03 09:45:00 -0.010530  0.840949    0.003595 -0.012937       NaN
  2017-01-03 10:00:00 -0.005218  1.191392    0.005506 -0.004742       NaN
  2017-01-03 10:15:00  0.004456  0.554745    0.004171 -0.001138       NaN
  2017-01-03 10:30:00 -0.001799  0.484170    0.004360  0.003315       NaN
  2017-01-03 10:45:00 -0.001767  0.742527    0.002465 -0.004975       NaN
  2017-01-03 11:00:00  0.000046  1.119380    0.003421  0.002896       NaN
  2017-01-03 11:15:00  0.004687  0.694956    0.001901  0.003925       NaN
  2017-01-03 11:30:00 -0.003899  0.841778    0.003386 -0.004027       NaN
  2017-01-03 11:45:00 -0.003412  1.025858    0.003048 -0.004419  1.158692
  2017-01-03 12:00:00  0.010763  0.669963    0.003621  0.010186  0.773829
  2017-01-03 12:15:00 -0.016368  0.639

In [235]:
window_size = 12
# The first window is from row 0 to 3, i.e. iloc[0: 0 + window_size - 1]
# The last window is from row len(x.index) - window_size + 1 to len(x.index)
# len(x.index) - window_size + 1 is 
#test.apply(lambda x: x.iloc[i: i + k] for i in range(len(x.index) - k + 1))
#test2 = test.apply(lambda x: x.iloc[i: i + window_size] for i in range(len(x.index) - window_size + 1))
#.apply(lambda x: pd.concat([x.iloc[i: i + k] for i in range(len(x.index) - k + 1)]))])
def make_more(df):
    return [df.iloc[i: i + window_size] for i in range(len(df.index) - window_size + 1)]

#test2 = test.apply(lambda x: make_more(df))

In [234]:
test4[0]

,return,volume,volatility,signal_A,signal_B
time,,,,,
2017-01-03 09:30:00,0.000000,1.357828,0.011823,NaN,NaN
2017-01-03 09:45:00,-0.010530,0.840949,0.003595,-0.012937,NaN
2017-01-03 10:00:00,-0.005218,1.191392,0.005506,-0.004742,NaN
2017-01-03 10:15:00,0.004456,0.554745,0.004171,-0.001138,NaN
2017-01-03 10:30:00,-0.001799,0.484170,0.004360,0.003315,NaN
2017-01-03 10:45:00,-0.001767,0.742527,0.002465,-0.004975,NaN
2017-01-03 11:00:00,0.000046,1.119380,0.003421,0.002896,NaN
2017-01-03 11:15:00,0.004687,0.694956,0.001901,0.003925,NaN
2017-01-03 11:30:00,-0.003899,0.841778,0.003386,-0.004027,NaN


In [132]:
df = pd.DataFrame({'user' : ['A', 'A', 'A', 'B', 'B', 'B','B'],
                  'attritube1' : [0,1,1,1,0,2,9], 
                  'attritube2':[1,2,3,3,0,0,1]})

In [134]:
df

,user,attritube1,attritube2
0,A,0,1
1,A,1,2
2,A,1,3
3,B,1,3
4,B,0,0
5,B,2,0
6,B,9,1


In [149]:
k=3
test3 = df.groupby('user').apply(lambda x: x.iloc[i: i + k] for i in range(len(x.index) - k + 1))

NameError: name 'x' is not defined

In [148]:
test3[0]

[  user  attritube1  attritube2
 0    A           0           1
 1    A           1           2
 2    A           1           3]

In [131]:
test2

,return,volume,volatility,signal_A,signal_B
time,,,,,
2017-01-03 09:30:00,0.000000,1.357828,0.011823,NaN,NaN
2017-01-03 09:45:00,-0.010530,0.840949,0.003595,-0.012937,NaN
2017-01-03 10:00:00,-0.005218,1.191392,0.005506,-0.004742,NaN
2017-01-03 10:15:00,0.004456,0.554745,0.004171,-0.001138,NaN
2017-01-03 10:30:00,-0.001799,0.484170,0.004360,0.003315,NaN
2017-01-03 10:45:00,-0.001767,0.742527,0.002465,-0.004975,NaN
2017-01-03 11:00:00,0.000046,1.119380,0.003421,0.002896,NaN
2017-01-03 11:15:00,0.004687,0.694956,0.001901,0.003925,NaN
2017-01-03 11:30:00,-0.003899,0.841778,0.003386,-0.004027,NaN
